# Eksempel på agent for bestilling av hotell og fly

Denne løsningen vil hjelpe deg med å bestille flybilletter og hotell. Scenariet er en reise fra London Heathrow LHR 20. februar 2024 til New York JFK med retur 27. februar 2025, med økonomiklasse og kun med British Airways. Jeg ønsker å bo på et Hilton-hotell i New York, vennligst oppgi kostnader for fly og hotell.


# Initialiser Azure AI Agent Service og hent konfigurasjonsinformasjon fra **.env**

### **.env** 

Opprett en .env-fil 

**.env** inneholder tilkoblingsstrengen til Azure AI Agent Service, modellen som brukes av AOAI, og den tilsvarende Google API Search-tjenesten API, ENDPOINT, osv.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Navnet på din Azure AI Agent Service Model Deployment"

[**NOTE**] Du vil trenge en modell med 100,000 Rate Limit (Tokens per minutt) og en Rate Limit på 600 (Forespørsler per minutt)

  Du kan få modellen i Azure AI Foundry - Model og Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Tilkoblingsstrengen til ditt Azure AI Agent Service-prosjekt"

  Du kan finne tilkoblingsstrengen til prosjektet i prosjektoversikten på AI ​​Foundry Portal-skjermen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

For å få Model Deployment Name og Project Connection String til Azure AI Agent Service, må du opprette Azure AI Agent Service. Det anbefales å bruke [denne malen](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) for å opprette den direkte （***Merk:*** Azure AI Agent Service er for øyeblikket satt til en begrenset region. Det anbefales at du refererer til [denne lenken](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) for å sette regionen)

Agenten må ha tilgang til SERPAPI. Det anbefales å registrere deg via [denne lenken](https://serpapi.com/searches). Etter registrering kan du få en unik API KEY og ENDPOINT.


# Oppsett

For å kjøre denne notatboken, må du sørge for at du har installert de nødvendige bibliotekene ved å kjøre `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Din Semantic Kernel-versjon bør være minst 1.27.2.


Last inn .env-filinnstillingene og ressursene dine, vennligst sørg for at du har lagt til nøklene og innstillingene dine og opprettet en lokal .env-fil.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Logg inn på Azure

Du må nå logge inn på Azure. Åpne en terminal og kjør følgende kommando:

```bash
az login
```

Denne kommandoen vil be deg om å oppgi dine Azure-legitimasjoner, slik at Azure AI Agent-tjenesten kan fungere riktig.


# Forklaring:
Dette er en variabel som lagrer API-nøkkelen for å få tilgang til en SERP (Search Engine Results Page) API-tjeneste. En API-nøkkel er en unik identifikator som brukes til å autentisere forespørsler knyttet til kontoen din.

Formål: Formålet med denne linjen er å lagre API-nøkkelen i en variabel slik at den kan brukes til å autentisere forespørsler til SERP API-tjenesten. API-nøkkelen er nødvendig for å få tilgang til tjenesten og utføre søk.

Hvordan få en SERP API-nøkkel: For å få en SERP API-nøkkel, følg disse generelle trinnene på https://serpapi.com (de nøyaktige trinnene kan variere avhengig av hvilken SERP API-tjeneste du bruker):

Velg en SERP API-tjeneste: Det finnes flere SERP API-tjenester tilgjengelig, som SerpAPI, Google Custom Search JSON API og andre. Velg den som passer best til dine behov.

Registrer deg for en konto: Gå til nettsiden til den valgte SERP API-tjenesten og registrer deg for en konto. Du må kanskje oppgi grunnleggende informasjon og bekrefte e-postadressen din.

Opprett en API-nøkkel: Etter registrering, logg inn på kontoen din og naviger til API-seksjonen eller dashbordet. Se etter et alternativ for å opprette eller generere en ny API-nøkkel.  
Kopier API-nøkkelen til .env-filen din.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Forklaring:
BASE_URL: Dette er en variabel som lagrer basis-URL-en for SERP API-endepunktet. Variabelnavnet BASE_URL er en konvensjon som brukes for å indikere at denne URL-en er utgangspunktet for å sende API-forespørsler.  
'https://serpapi.com/search':  

Dette er den faktiske URL-strengen som er tilordnet BASE_URL-variabelen. Den representerer endepunktet for å utføre søk ved hjelp av SERP API.  

# Formål:
Formålet med denne linjen er å definere en konstant som holder basis-URL-en for SERP API. Denne URL-en vil bli brukt som utgangspunkt for å bygge API-forespørsler for å utføre søkeoperasjoner.  

# Bruk:
Ved å definere basis-URL-en i en variabel, kan du enkelt gjenbruke den i hele koden din når du trenger å sende forespørsler til SERP API. Dette gjør koden din mer vedlikeholdbar og reduserer risikoen for feil som oppstår ved å hardkode URL-en flere steder. Det nåværende eksempelet er https://serpapi.com/search?engine=bing, som bruker Bing søke-API. Ulike API-er kan velges på https://Serpapi.com  


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Forklaring:

Her er hvor plugin-koden din er plassert.

Klassedefinisjon: `class BookingPlugin`: Definerer en klasse kalt BookingPlugin som inneholder metoder for å bestille hoteller og flyreiser.

Metode for hotellbestilling:

- `@kernel_function(description="booking hotel")`: En dekoratør som beskriver funksjonen som en kjernefunksjon for hotellbestilling.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definerer en metode for hotellbestilling med annoterte parametere og returtype.

Metoden konstruerer en ordbok med parametere for hotellbestillingsforespørselen og sender en GET-forespørsel til SERP API. Den sjekker statusen på svaret og returnerer hotellinformasjonen hvis forespørselen var vellykket, eller None hvis forespørselen mislyktes.

Metode for flybestilling:

- `@kernel_function(description="booking flight")`: En dekoratør som beskriver funksjonen som en kjernefunksjon for flybestilling.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definerer en metode for flybestilling med annoterte parametere og returtype.

Metoden konstruerer ordbøker med parametere for forespørslene om utreise- og returflyvninger og sender GET-forespørsler til SERP API. Den sjekker statusen på svarene og legger til flyinformasjonen i resultatstrengen hvis forespørselen var vellykket, eller skriver ut en feilmelding hvis forespørselen mislyktes. Metoden returnerer resultatstrengen som inneholder flyinformasjonen.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Forklaring:
Importsetninger: Importer nødvendige moduler for Azure-legitimasjon, AI-agent, innhold i chatmeldinger, forfatterrolle og kernel-funksjonsdekorator.

Asynkron kontekstbehandler: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dette setter opp en asynkron kontekstbehandler for å håndtere Azure-legitimasjon og opprette en AI-agentklient.

Agentnavn og instruksjoner: 
- `AGENT_NAME = "BookingAgent"`: Definerer navnet på agenten.
- `AGENT_INSTRUCTIONS = """..."""`: Gir detaljerte instruksjoner til agenten om hvordan den skal håndtere bookingforespørsler.

Opprett agentdefinisjon: `agent_definition = await client.agents.create_agent(...)`: Oppretter en agentdefinisjon med den spesifiserte modellen, navnet og instruksjonene.

Opprett AzureAI-agent: `agent = AzureAIAgent(...)`: Oppretter en AzureAI-agent ved hjelp av klienten, agentdefinisjonen og den definerte pluginen.

Opprett tråd: `thread: AzureAIAgentThread | None = None`: Opprett en tråd for agenten. Det er ikke nødvendig å først opprette en tråd - hvis verdien `None` er angitt, vil en ny tråd bli opprettet under første kall og returnert som en del av responsen.

Brukerinndata: `user_inputs = ["..."]`: Definerer en liste med brukerinndata som agenten skal behandle.

I finally-blokken, slett tråden og agenten for å frigjøre ressurser.


# Autentisering

Klassen `DefaultAzureCredential` er en del av Azure SDK for Python. Den gir en standard måte å autentisere med Azure-tjenester på. Den forsøker å autentisere ved hjelp av flere metoder i en spesifikk rekkefølge, som miljøvariabler, administrert identitet og Azure CLI-legitimasjon.

Asynkrone operasjoner: aio-modulen indikerer at klassen DefaultAzureCredential støtter asynkrone operasjoner. Dette betyr at du kan bruke den med asyncio for å utføre ikke-blokkerende autentiseringsforespørsler.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
